In [1]:
# Install packages 
import os
import pandas as pd
import time
import random
from transformers import pipeline
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import undetected_chromedriver as uc
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# URL of Forex Factory News
chrome_options = Options()
url = "https://www.forexfactory.com/news"
chrome_options.add_argument("--disable-blink-features=AutomationControlled") 

In [ ]:
ua = UserAgent()
chrome_options.add_argument(f"user-agent={ua.random}")

In [ ]:
driver = uc.Chrome()
driver.get(url)
time.sleep(3)

In [ ]:
driver.get("https://www.forexlive.com")

In [ ]:
# Number of times to attempt clicking the button
max_attempts = 50

for attempt in range(max_attempts):
    try:
        print(f"Attempt {attempt + 1}/{max_attempts}")

        # Wait for the button to be clickable
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'a-see-more-less all-news__see-more-less all-news__see-more-less')]"))
        )

        # Add a random delay
        time.sleep(3)

        # Scroll to the button
        driver.execute_script("arguments[0].scrollIntoView();", more_button)
        
        # Add another random delay
        time.sleep(3)

        # Click the button
        driver.execute_script("arguments[0].click();", more_button)
        
        # Random delay after clicking
        time.sleep(3)

        response = driver.page_source

    except (TimeoutException, NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Exception occurred on attempt {attempt + 1}: {e}")
        print("Button is no longer available or page has changed. Stopping script.")
        break  # Exit the loop when the button is not found or not clickable

In [ ]:
soup = BeautifulSoup(response, "html.parser")
articles=[]
for var in soup.find_all('a' , attrs={'data-v-5186c092': True}):
    articles.append(var.get_text(strip=True))

In [ ]:
# Load FinBERT model
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert")
# Function to classify sentiment using FinBERT
def classify_sentiment(text):
    result = finbert(text)[0] 
    return result["label"]  

In [ ]:
df = pd.read_csv("data/forex_news.csv")
df = pd.concat([df, pd.DataFrame({'news': articles, 'labels': list(map(classify_sentiment, articles))})], axis=0, ignore_index=True)
df = df.drop_duplicates().reset_index(drop=True)  

In [ ]:
df['news'] = df['news'].fillna('')
df=df[['news', 'labels']]
df.to_csv("data/forex_news.csv")